In [1]:
# !pip install kafka-python
# !pip install pyspark==2.4.5
## EXECUTION STEPS ###
#Steps to execute:
'''
1. Start a zookeeper server on port 2181 (default config port)

2. Start a kafka broker on port 9092 (default config port)

3. Execute the cells as it is on jupyter

4. Run a kafka client on port localhost:9092 and topic mall and verify data is streaming from sensor.

5. Now run another kafka consumer client on port localhost:9092 for topic  offers 

6. keep mallcustomer.csv and shopcoupons.csv in the same dir as the ipynb file'''


In [1]:
from random import randint
import threading
import datetime
from json import dumps
from kafka import KafkaProducer
from kafka.admin import KafkaAdminClient, NewTopic
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SQLContext, Row

In [2]:
# Connect to kafka broker
admin_client = KafkaAdminClient(
    bootstrap_servers="localhost:9092",
    client_id='test'
)

In [3]:
# Create a producer on the vonnected kafka broker
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: dumps(x).encode('utf-8'))

## Excercise 1 - Get data from and send stream to kafka

In [ ]:
# Function to generate sensor data and send it to kafka producer
def get_sensor_data():
    threading.Timer(1.0, get_sensor_data).start()
    customer_id = randint(1, 200)
    shop_id = randint(1, 10)
    time = datetime.datetime.now().strftime("%H:%M:%S")
    data = {'customer_id': customer_id, 'shop_id': shop_id, 'timestamp': time}
    print("sending data"+str(data))
    producer.send('mall', value=str(data))

In [5]:
# Create a topic named mall if one already doesnot exist and send data to that topic
topic_name = 'mall'

if topic_name not in admin_client.list_topics():
    topic_list = [NewTopic(name="mall", num_partitions=1, replication_factor=1)]
    admin_client.create_topics(new_topics=topic_list, validate_only=False)

get_sensor_data()

## Exercise 3 - Process the stream from kafka and select suitable offers and send to another kafka topic to which downstream app is connected to send sms

In [6]:
# create a spark context
sc = SparkContext(appName="Python Streaming Direct Kafka Mall Offers")

# create a streaming context
ssc = StreamingContext(sc, 1)

# designate broker and topic from which stream is to be read
brokers, topic = "localhost:9092", "mall"

# create some global variables required later
gender = ''
spending_score = ''
cust_id = ''

# create sql context using pyspark
sqlContext = SQLContext(sc)

In [7]:
# Create a customer dataset and offers dataset from respective csvs
customer_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true') \
    .load('MallCustomers.csv')

offers_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true') \
    .load('ShopCoupons.csv')

customer_df.registerTempTable('customer_data')

offers_df.registerTempTable('offers_data')

In [10]:
# Function to process the rdd which contains customer_id in the preprocessed stream
# The preprocessed stream here is joined with the customer dataset from which we find 
# the gender and the spending score of the customer and save them in the global variables
# created above.
def process_cust(time, rdd):
    try:
        # Convert RDD[String] to RDD[Row] to DataFrame
        rowRdd = rdd.map(lambda id: Row(c_id=id))
        idDataFrame = sqlContext.sparkSession.createDataFrame(rowRdd)

        # Creates a temporary view using the DataFrame.
        idDataFrame.createOrReplaceTempView("ids")

        # Extract data from temp view and save it to global variables
        idsDataFrame = sqlContext.sql('select * from ids')

        global cust_id

        cust_id = idsDataFrame.head().__getitem__('c_id')

        global gender

        gender = sqlContext.sql('select gender from customer_data where Customer_ID={}'
                                .format(cust_id)) \
            .head().__getitem__('gender')

        global spending_score

        spending_score = sqlContext.sql('select spending_score from customer_data where Customer_ID={}'
                                        .format(cust_id)) \
            .head().__getitem__('spending_score')

    except:

        pass

In [9]:
# Function to join the preprocessed stream with offers dataset
# This function uses the store_id of a mall and the global variables
# (spending_score and gender) to filter out the best offer for the 
# customer and send it to designated kafka topic connected to downstream application.
# If the processor finds no suitable offer for a customer it sends nothing to kafka,
# but prints a suitable message for user understanding.
def process_shop(time, rdd):
    print("==========%s=========" % str(time))

    try:
        # Convert RDD[String] to RDD[Row] to DataFrame
        rowRdd = rdd.map(lambda id: Row(s_id=id))
        idDataFrame = sqlContext.sparkSession.createDataFrame(rowRdd)

        # Creates a temporary view using the DataFrame.
        idDataFrame.createOrReplaceTempView("ids")

        # Extract data from temp view and use it to filter the offers out
        idsDataFrame = sqlContext.sql('select * from ids')

        sh_id = idsDataFrame.head().__getitem__('s_id')

        offer = sqlContext.sql('select offer_details, spending_score_range, gender from offers_data where shop_id={}'
                               .format(sh_id)) \
            .head(5)

        shop_name = sqlContext.sql('select store_name from offers_data where shop_id={}'
                                   .format(sh_id)) \
            .head().__getitem__('store_name')

        offers = []
        genders = []
        spending_scores = []
        offer_to_be_sent = 'NA'

        for item in offer:
            offers.append(item.__getitem__('offer_details'))
            spending_scores.append(item.__getitem__('spending_score_range'))
            genders.append(item.__getitem__('gender'))

        for score_range in spending_scores:
            sp_score_init = int(score_range.split('-')[0])
            sp_score_final = int(score_range.split('-')[1])

            if int(spending_score) in range(sp_score_init, sp_score_final):

                if str(genders[(spending_scores.index(score_range))]).lower() == str(gender).lower() or \
                        str(genders[(spending_scores.index(score_range))]).lower() == 'all':
                    offer_to_be_sent = offers[(spending_scores.index(score_range))]

        if offer_to_be_sent != 'NA':
            offers_data = 'customer-id: {}, store: {}, offer: {}'.format(cust_id, shop_name, offer_to_be_sent)

            producer.send('offers', value=offers_data)
        else:
            print('No offers applicable for the customer {} in {} store'.format(cust_id, shop_name))

    except:

        pass

In [ ]:
topic_name = 'offers'

if topic_name not in admin_client.list_topics():
    topic_list = [NewTopic(name="offers", num_partitions=1, replication_factor=1)]
    admin_client.create_topics(new_topics=topic_list, validate_only=False)

# Read the kafka stream for incoming messages and store in a variable
kvs = KafkaUtils.createDirectStream(ssc, [topic], {"metadata.broker.list": brokers})

# Pre process the message to shape it into required format 
stream_data = kvs.map(lambda x: eval(x[1]).replace('{', '').replace('}', ''))

# Extract the Customer information from the stream
customer_id = stream_data.map(lambda y: y.split(',')[0].split(': ')[1])

# For each extracted customer info run the customer processor function on it
customer_id.foreachRDD(process_cust)

# Extract the information about the nearest store to the customer
shop_id = stream_data.map(lambda y: y.split(',')[1].split(': ')[1])

# For each extracted info run the designated processor function on it
shop_id.foreachRDD(process_shop)

ssc.start()

ssc.awaitTermination()

==========2021-01-15 19:22:18=========
==========2021-01-15 19:22:19=========
No offers applicable for the customer 167 in Caretlane store
==========2021-01-15 19:22:20=========
==========2021-01-15 19:22:21=========
==========2021-01-15 19:22:22=========
==========2021-01-15 19:22:23=========
No offers applicable for the customer 23 in Shoppers Stop store
==========2021-01-15 19:22:24=========
==========2021-01-15 19:22:25=========
==========2021-01-15 19:22:26=========
No offers applicable for the customer 114 in Reliance Trendz store
==========2021-01-15 19:22:27=========
==========2021-01-15 19:22:28=========
No offers applicable for the customer 45 in PVR store
==========2021-01-15 19:22:29=========
No offers applicable for the customer 81 in Lenskart store
==========2021-01-15 19:22:30=========
==========2021-01-15 19:22:31=========
==========2021-01-15 19:22:32=========
==========2021-01-15 19:22:33=========
==========2021-01-15 19:22:34=========
==========2021-01-15 19:22:35===